ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
'c:\Users\derne\OneDrive' is not recognized as an internal or external command,
operable program or batch file.
'c:\Users\derne\OneDrive' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
!pip install -U pip
!pip install -U codeium-jupyter

In [2]:
import MetaTrader5 as mt5
import pandas as pd
import pandas_ta as ta





In [3]:

if not mt5.initialize(): 
    print( 'initialization failed')
    quit() 
else: print('initialization success') 

login = 192918303
password = "Demo@1234"
server = "Exness-MT5Trial"
# mt5.login(login, password, server)

if not mt5.login(login, password, server):
    print("Failed to connect to the server")
else:
    print("Connected to the server successfully")
    print(mt5.account_info())


initialization success
Connected to the server successfully
AccountInfo(login=192918303, trade_mode=0, leverage=200, limit_orders=1024, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=10745.11, credit=0.0, profit=-2304.55, equity=8440.56, margin=1136.4, margin_free=7304.16, margin_level=742.7455121436113, margin_so_call=60.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DemoBotTester', server='Exness-MT5Trial', currency='USD', company='Exness Technologies Ltd')


In [4]:
df = mt5.copy_rates_from_pos('BTCUSDm', mt5.TIMEFRAME_M30, 0, 90000 )
df = pd.DataFrame(df)
df['time']= pd.to_datetime(df['time'], unit='s')
#  rename columns o fit backtesting parameters
df = df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'tick_volume': 'Volume'})

df.set_index('time', inplace=True)


In [6]:
df

,Open,High,Low,Close,Volume,spread,real_volume
time,,,,,,,
2019-12-02 16:30:00,7308.65,7308.65,7256.68,7288.26,1749,867,0
2019-12-02 17:00:00,7288.70,7289.62,7252.61,7269.89,1746,887,0
2019-12-02 17:30:00,7269.92,7301.77,7268.63,7287.20,1731,877,0
2019-12-02 18:00:00,7287.10,7287.58,7246.42,7266.53,1737,865,0
2019-12-02 18:30:00,7266.71,7286.11,7253.34,7278.92,1718,844,0
...,...,...,...,...,...,...,...
2025-01-20 00:30:00,100953.06,101096.12,99396.96,99659.70,15636,2880,0
2025-01-20 01:00:00,99659.70,100333.06,99469.72,100141.76,13829,2880,0
2025-01-20 01:30:00,100141.76,100832.64,99785.94,100536.88,11103,2880,0


In [7]:
import pandas_ta as ta
df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df["EMA"]=ta.ema(df.Close, length=100)
df.head()

,Open,High,Low,Close,Volume,spread,real_volume,VWAP,EMA
time,,,,,,,,,
2019-12-02 16:30:00,7308.65,7308.65,7256.68,7288.26,1749,867,0,7284.530000,NaN
2019-12-02 17:00:00,7288.70,7289.62,7252.61,7269.89,1746,887,0,7277.624266,NaN
2019-12-02 17:30:00,7269.92,7301.77,7268.63,7287.20,1731,877,0,7280.354384,NaN
2019-12-02 18:00:00,7287.10,7287.58,7246.42,7266.53,1737,865,0,7276.983898,NaN
2019-12-02 18:30:00,7266.71,7286.11,7253.34,7278.92,1718,844,0,7276.153911,NaN


In [8]:
emasignal = [0]*len(df)
backcandles = 6

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.EMA[i]:
            dnt=0
        if df.Low[i]<=df.EMA[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        emasignal[row]=3
    elif upt==1:
        emasignal[row]=2
    elif dnt==1:
        emasignal[row]=1

df['EMASignal'] = emasignal

C:\Users\derne\AppData\Local\Temp\ipykernel_62780\3793931515.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.High[i]>=df.EMA[i]:
C:\Users\derne\AppData\Local\Temp\ipykernel_62780\3793931515.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.Low[i]<=df.EMA[i]:


# VWAP SIGNAL
checks if the High and Low prices are above or below the VWAP over the past backcandles rows.
- If both conditions are met (High above VWAP and Low below VWAP), it assigns a signal of 3 to VWAPsignal.
- If only the High is above VWAP, it assigns a signal of 2.
- If only the Low is below VWAP, it assigns a signal of 1.

In [9]:
VWAPsignal = [0]*len(df)
backcandles = 3

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if df.High[i]>=df.VWAP[i]:
            dnt=0
        if df.Low[i]<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        #print("!!!!! check trend loop !!!!")
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal

C:\Users\derne\AppData\Local\Temp\ipykernel_62780\3867349253.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.High[i]>=df.VWAP[i]:
C:\Users\derne\AppData\Local\Temp\ipykernel_62780\3867349253.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df.Low[i]<=df.VWAP[i]:


In [10]:
def TotalSignal(l):
    myclosedistance = 100
    if (df.EMASignal[l]==2 and df.VWAPSignal[l]==2#and df.EngulfingSignal[l]==2 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 2
    if (df.EMASignal[l]==1 and df.VWAPSignal[l]==1#and df.EngulfingSignal[l]==1 
        and min(abs(df.VWAP[l]-df.High[l]),abs(df.VWAP[l]-df.Low[l]))<=myclosedistance):
            return 1
        
TotSignal = [0]*len(df)
for row in range(0, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

C:\Users\derne\AppData\Local\Temp\ipykernel_62780\168514811.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df.EMASignal[l]==2 and df.VWAPSignal[l]==2#and df.EngulfingSignal[l]==2
C:\Users\derne\AppData\Local\Temp\ipykernel_62780\168514811.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (df.EMASignal[l]==1 and df.VWAPSignal[l]==1#and df.EngulfingSignal[l]==1
C:\Users\derne\AppData\Local\Temp\ipykernel_62780\168514811.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). T

In [11]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-3
    elif x['TotalSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[1500:1900]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=1), name="EMA"),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, line=dict(color='blue', width=1), name="VWAP")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()

In [14]:
dfpl = df[:].copy()
dfpl.reset_index(inplace=True)
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=5)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

# Backtesting 

- If the signal is 2 and there are no open trades, it buys with a stop loss (sl1) and take profit (tp1) set based on the ATR (Average True Range).
- If the signal is 1 and there are no open trades, it sells with a stop loss and take profit set similarly.
- The strategy is then backtested on a dataset dfpl with an initial cash balance of $100,000, a margin of 1/5, and no commission.

In [15]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 0.8*self.data.ATR[-1]
        TPSLRatio = 2

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100000, margin=1/5, commission=.00)
stat = bt.run()
stat 

C:\Users\derne\AppData\Local\Temp\ipykernel_62780\2396171223.py:26: UserWarning:

Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).

C:\Users\derne\AppData\Local\Temp\ipykernel_62780\2396171223.py:26: UserWarning:

Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.



Start                                     0.0
End                                   89999.0
Duration                              89999.0
Exposure Time [%]                   12.977778
Equity Final [$]                  57957.41193
Equity Peak [$]                 242205.405594
Return [%]                         -42.042588
Buy & Hold Return [%]             1292.498484
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -82.892155
Avg. Drawdown [%]                  -11.596685
Max. Drawdown Duration                45187.0
Avg. Drawdown Duration            2883.870968
# Trades                               2480.0
Win Rate [%]                        33.830645
Best Trade [%]                       3.536652
Worst Trade [%]                     -4.442221
Avg. Trade [%]                    

In [16]:
bt.plot()

GridPlot(id='p1342', ...)